In [184]:
import cobra
from cobra.io import (
    load_json_model,
    save_json_model,
    load_matlab_model,
    save_matlab_model,
    read_sbml_model,
    write_sbml_model,
)
from cobra.flux_analysis import flux_variability_analysis

In [185]:
M_xanthus = read_sbml_model("../M_xanthus_model.sbml")
M_xanthus

Name,myxo_model
Memory address,7759a37fa210
Number of metabolites,1280
Number of reactions,1367
Number of genes,1192
Number of groups,0
Objective expression,1.0*OF_BIOMASS - 1.0*OF_BIOMASS_reverse_80d2e
Compartments,"c, e"


In [186]:
E_coli = read_sbml_model("../E_coli_model.sbml")
E_coli

Name,iML1515
Memory address,7759a37fbed0
Number of metabolites,1877
Number of reactions,2714
Number of genes,1516
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


In [ ]:
print("E_coli: " + str(E_coli.compartments))
print(
    "M_xanthus: " + str(M_xanthus.compartments)
)  # Both Gram-, both should have periplasm

E_coli: {'c': 'cytosol', 'e': 'extracellular space', 'p': 'periplasm'}
M_xanthus: {'c': '', 'e': ''}


In [21]:
pFBA = cobra.flux_analysis.pfba(M_xanthus)
FBA = M_xanthus.optimize()
FVA = flux_variability_analysis(M_xanthus, M_xanthus.reactions[:10])
print(FBA.objective_value, pFBA.objective_value)

70.09970088263155 51953.14196186318


In [4]:
M_xanthus.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Fe3_e,EX_Fe3_e,0.3365,0,0.00%
acac_e,EX_acac_e,348.5,4,8.47%
acald_e,EX_acald_e,1000,2,12.16%
ca2_e,EX_ca2_e,0.1122,0,0.00%
cgly_e,EX_cgly_e,0.1122,5,0.00%
cl_e,EX_cl_e,0.1122,0,0.00%
cobalt2_e,EX_cobalt2_e,0.1122,0,0.00%
cu2_e,EX_cu2_e,0.1122,0,0.00%
etoh_e,EX_etoh_e,14.28,2,0.17%
fru_e,EX_fru_e,12.4,6,0.45%


In [13]:
E_coli.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.06116,0,0.00%
cl_e,EX_cl_e,0.06116,0,0.00%
cobalt2_e,EX_cobalt2_e,0.0002938,0,0.00%
cu2_e,EX_cu2_e,0.008332,0,0.00%
fe2_e,EX_fe2_e,0.1887,0,0.00%
glc__D_e,EX_glc__D_e,105.7,6,100.00%
k_e,EX_k_e,2.294,0,0.00%
mg2_e,EX_mg2_e,0.1019,0,0.00%
mn2_e,EX_mn2_e,0.00812,0,0.00%
mobd_e,EX_mobd_e,8.226E-05,0,0.00%


In [12]:
E_coli.reactions.EX_pydx_e #1
E_coli.reactions.EX_spmd_e #2
#E_coli.reactions.EX_leu_L_e #4
E_coli.reactions.EX_cl_e #3
#E_coli.reactions.EX_metox_e 
E_coli.reactions.EX_cobalt2_e
#E_coli.reactions.EX_ile_L_e  #5

Reaction identifier,EX_cobalt2_e
Name,Co2+ exchange
Memory address,0x73e5b41712e0
Stoichiometry,cobalt2_e <=> Co2+ <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


**ADD REACTION**

In [ ]:
#TODO add exchange reaction of importing mtabolite used for BIOMASS prod: PR_BIOMASS
M_xanthus

#TODO add exchange reaction of exporting BIOMASS: PR_BIOMASS
E_coli

#TODO add constrains to one model to force it to have a same flux as other model

Reaction identifier,BIOMASS_Ec_iML1515_WT_75p37M
Name,E. coli biomass objective function (iML1515) - WT - with 75.37 GAM estimate
Memory address,0x73e5af2df070
Stoichiometry,0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c +... 0.000223 10-Formyltetrahydrofolate + 0.000223 2-Demethylmenaquinol 8 + 2.5e-05 [2Fe-2S] iron-sulfur cluster + 0.000248 [4Fe-4S] iron-sulfur cluster + 0.000223 5-Methyltetrahydrofolate + 0.000279...
GPR,
Lower bound,0.0
Upper bound,1000.0


For Escherichia coli

In [27]:
E_coli.reactions.BIOMASS_Ec_iML1515_WT_75p37M.reaction

'0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 70.028756 h2o_c + 0.000223 hemeO_c + 0.092056 his__L_c + 0.282306 ile__L_c + 0.18569 k_c + 0.437778 leu__L_c + 3e-06 lipopb_c + 0.333448 lys__L_c + 3.1e-05 malcoa_c + 0.149336 met__L_c + 0.008253 mg2_c + 0.000223 mlthf_c + 0.000658 mn2_c + 7e-06 mobd_c + 

In [53]:
E_coli.metabolites.glu__L_c

Metabolite identifier,glu__L_c
Name,L-Glutamate
Memory address,0x73e5b5cd3430
Formula,C5H8NO4
Compartment,c
In 51 reaction(s),"GGPTRCS, HSTPT, ACGS, ACOTA, IG3PS, BIOMASS_Ec_iML1515_WT_75p37M, GLUN, SOTA, GLUDy, P5CD, SGDS, VALTA, GLUPRT, GLUSy, CTPS2, GLUt4pp, GLUR, ASPTA, ASNS1, GMPS2, OHPBAT, DHFS, GF6PTA, UDPKAAT,..."


In [56]:
Ereaction = cobra.Reaction('PR_BIOMASS')
Ereaction.name = 'Predation Biomass'
Ereaction.subsystem = ''
Ereaction.lower_bound = 0.  # This is the default
Ereaction.upper_bound = 1000.  # This is the default

In [57]:
adp_c = E_coli.metabolites.adp_c
h_c = E_coli.metabolites.h_c
pi_c = E_coli.metabolites.pi_c
ppi_C = E_coli. metabolites.ppi_c

Ereaction.add_metabolites({
    adp_c: -75.37723,
    h_c: -75.37723,
    pi_c: -75.37323,
    ppi_C: -0.749831
})

In [58]:
Ereaction.reaction

'75.37723 adp_c + 75.37723 h_c + 75.37323 pi_c + 0.749831 ppi_c --> '

In [25]:
Ereaction.gene_reaction_rule = ''
Ereaction.genes

frozenset()

In [ ]:
#E_coli.add_reactions([Ereaction])

for Myxococcus xanthus

In [50]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [51]:
M_xanthus.metabolites.PROTEIN_W_c

Metabolite identifier,PROTEIN_W_c
Name,Protein [c]
Memory address,0x73e5b67cfb10
Formula,C4H5N2O3R2
Compartment,c
In 2 reaction(s),"OF_BIOMASS, OF_PROTEIN"


In [ ]:
Mreaction = cobra.Reaction('PR_BIOMASS')
Mreaction.name = 'Predation Biomass'
Mreaction.subsystem = ''
Mreaction.lower_bound = 0.  # This is the default
Mreaction.upper_bound = 1000.  # This is the default

In [ ]:
adp_c = M_xanthus.metabolites.adp_c #used end OF
h_c = M_xanthus.metabolites.h_c #used end OF
pi_c = M_xanthus.metabolites.pi_c #used end OF
ppi_C = M_xanthus. metabolites.ppi_c

Mreaction.add_metabolites({
    adp_c: 75.37723,
    h_c: 75.37723,
    pi_c: 75.37323,
    ppi_C: 0.749831
})

In [46]:
Mreaction.reaction

' --> 75.37723 adp_c + 75.37723 h_c + 75.37323 pi_c + 0.749831 ppi_c'

In [47]:
Mreaction.gene_reaction_rule = ''
Mreaction.genes

frozenset()

In [ ]:
#M_xanthus.add_reactions([Mreaction])

Add constraints to for those to have the same flux

In [ ]:
#same_flux = E_coli.problem.Constraint(
#    M_xanthus.reactions.PR_BIOMASS.flux_expression - E_coli.reactions.PR_BIOMASS.flux_expression,
#    lb=0,
#    ub=0)
#E_coli.add_cons_vars(same_flux)

**METHOD 4**

In [124]:
Biomass_c = cobra.Metabolite(
    'Biomass_c',
    formula='BIOMASS',
    name='Biomass [c]',
    compartment='c')

Biomass_e = cobra.Metabolite(
    'Biomass_e',
    formula='BIOMASS',
    name='Biomass [e]',
    compartment='e')

M_xanthus.add_metabolites(Biomass_e) # add M_xanthus.metabolites.Biomass_e

Add both reactions: Biomass_e --> metabolites and EX_Biomass_e

In [125]:
Mreaction = cobra.Reaction('PR_BIOMASS')
Mreaction.name = 'Predation Biomass'
Mreaction.subsystem = ''
Mreaction.lower_bound = 0.  # This is the default
Mreaction.upper_bound = 1000.  # This is the default

In [126]:
M_xanthus.reactions.OF_BIOMASS.reaction

'0.00160004066244054 10fthf_c + 0.00160004066244054 2dmmq8_c + 0.00160004066244054 5mthf_c + 0.00160004066244054 ACP_c + 0.00160004066244054 Fe2_c + 0.00160004066244054 Fe3_c + 0.5441666 PROTEIN_W_c + 0.032079293220334 Peptidoglycan_polymer_n_subunits_c + 0.1911666 RNA_W_c + 0.0248034535317025 Stearoylcardiolipin__c + 0.00160004066244054 adocbl_c + 0.00160004066244054 amet_c + 40.0 atp_c + 0.00160004066244054 ca2_c + 0.00160004066244054 cl_c + 0.00160004066244054 coa_c + 0.00160004066244054 cobalt2_c + 0.00160004066244054 cu2_c + 0.0121667 dna_c + 0.00160004066244054 fad_c + 0.00160004066244054 gthrd_c + 40.0 h2o_c + 0.00160004066244054 k_c + 0.032079293220334 kdo2lipid4L_c + 0.00160004066244054 mg2_c + 0.00160004066244054 mn2_c + 0.00160004066244054 mqn8_c + 0.00160004066244054 nad_c + 0.00160004066244054 nadp_c + 0.0248034535317025 pe180_c + 0.0248034535317025 pg180_c + 0.00160004066244054 pheme_c + 0.00160004066244054 ptrc_c + 0.00160004066244054 pydx5p_c + 0.00160004066244054 q8_c 

In [127]:
Mreaction = cobra.Reaction('PR_BIOMASS')
Mreaction.name = 'Predation Biomass'
Mreaction.subsystem = ''
Mreaction.lower_bound = 0.  # This is the default
Mreaction.upper_bound = 1000.  # This is the default

Biomass_e = M_xanthus.metabolites.Biomass_e

spmd_e = M_xanthus.metabolites.spmd_e # carbon source from exchange used in biomass reaction
pheme_e = M_xanthus.metabolites.pheme_e
val_L_e = M_xanthus.metabolites.val_L_e # imported AA
ile_L_e = M_xanthus.metabolites.ile_L_e
leu_L_e = M_xanthus.metabolites.leu_L_e
lys_L_e = M_xanthus.metabolites.lys_L_e
his_L_e = M_xanthus.metabolites.his_L_e
gly_cys_L_e = M_xanthus.metabolites.gly_cys_L_e
cgly_e = M_xanthus.metabolites.cgly_e
pro_L_e = M_xanthus.metabolites.pro_L_e
alaala_e = M_xanthus.metabolites.alaala_e
glu_L_c = M_xanthus.metabolites.glu_L_c
met_L_e = M_xanthus.metabolites.met_L_e
thr_L_e = M_xanthus.metabolites.thr_L_e
fru_e = M_xanthus.metabolites.fru_e # import fructose
malt_e = M_xanthus.metabolites.malt_e
gam_e = M_xanthus.metabolites.gam_e # from here test
fum_e = M_xanthus.metabolites.fum_e
orn_e = M_xanthus.metabolites.orn_e
succ_e = M_xanthus.metabolites.succ_e
acald_e = M_xanthus.metabolites.acald_e

Mreaction.add_metabolites({
    Biomass_e: -1,
    spmd_e: 1,
    pheme_e: 1,
    val_L_e: 1,
    ile_L_e: 1,
    leu_L_e: 1,
    lys_L_e: 1,
    his_L_e: 1,
    gly_cys_L_e: 1,
    cgly_e: 1,
    pro_L_e: 1,
    alaala_e: 1,
    fru_e: 1,
    malt_e: 1,
    glu_L_c: 1,
    met_L_e: 1,
    thr_L_e: 1,
    gam_e: 1,
    fum_e: 1,
    orn_e: 1,
    succ_e: 1,
    acald_e: 1
}) # add the metabolites to the reaction

Mreaction.gene_reaction_rule = '' # define which gene allow the reaction 

M_xanthus.add_reactions([Mreaction]) # add the PR_Biomass reaction

M_xanthus.reactions.PR_BIOMASS

Reaction identifier,PR_BIOMASS
Name,Predation Biomass
Memory address,0x7759c1da87e0
Stoichiometry,Biomass_e --> acald_e + alaala_e + cgly_e + fru_e + fum_e + gam_e + glu_L_c + gly_cys_L_e + his_L_e + ile_L_e + leu_L_e + lys_L_e + malt_e + met_L_e + orn_e + pheme_e + pro_L_e + spmd_e + succ_e +... Biomass [e] --> Acetaldehyde [e] + Ala-Ala [e] + Cys-Gly [e] + D-Fructose [e] + Fumarate [e] + GLUM [e] + L-Glutamate [c] + Gly-Cys [e] + L-Histidine [e] + L-Isoleucine [e] + L-Leucine [e] +...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [128]:
Mreaction.reaction

'Biomass_e --> acald_e + alaala_e + cgly_e + fru_e + fum_e + gam_e + glu_L_c + gly_cys_L_e + his_L_e + ile_L_e + leu_L_e + lys_L_e + malt_e + met_L_e + orn_e + pheme_e + pro_L_e + spmd_e + succ_e + thr_L_e + val_L_e'

In [129]:
Mreaction.gene_reaction_rule = ''
Mreaction.genes

frozenset()

In [130]:
M_xanthus.add_reactions([Mreaction]) # add the PR_Biomass reaction
M_xanthus.add_boundary(M_xanthus.metabolites.get_by_id("Biomass_e"), type="exchange") #add the exchange reaction for Biomass_e

Ignoring reaction 'PR_BIOMASS' since it already exists.


Reaction identifier,EX_Biomass_e
Name,Biomass [e] exchange
Memory address,0x7759c2d2e780
Stoichiometry,Biomass_e <=> Biomass [e] <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [131]:
M_xanthus.reactions.EX_spmd_e.lower_bound = 0 # define the bound to 0 for metabolites in the reactions
M_xanthus.reactions.EX_pheme_e.lower_bound = 0
M_xanthus.reactions.EX_val_L_e.lower_bound = 0
M_xanthus.reactions.EX_ile_L_e.lower_bound = 0
M_xanthus.reactions.EX_leu_L_e.lower_bound = 0
M_xanthus.reactions.EX_lys_L_e.lower_bound = 0
M_xanthus.reactions.EX_gly_cys_L_e.lower_bound = 0
M_xanthus.reactions.EX_cgly_e.lower_bound = 0
M_xanthus.reactions.EX_pro_L_e.lower_bound = 0
M_xanthus.reactions.EX_alaala_e.lower_bound = 0
M_xanthus.reactions.EX_glu_L_e.lower_bound = 0
M_xanthus.reactions.EX_met_L_e.lower_bound = 0
M_xanthus.reactions.EX_thr_L_e.lower_bound = 0
M_xanthus.reactions.EX_fru_e.lower_bound = 0
M_xanthus.reactions.EX_gam_e.lower_bound = 0 # test
M_xanthus.reactions.EX_fum_e.lower_bound = 0
M_xanthus.reactions.EX_orn_e.lower_bound = 0
M_xanthus.reactions.EX_malt_e.lower_bound = 0
M_xanthus.reactions.EX_succ_e.lower_bound = 0
M_xanthus.reactions.EX_acald_e.lower_bound = 0

In [132]:
M_xanthus.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Biomass_e,EX_Biomass_e,737.4,0,0.00%
Fe3_e,EX_Fe3_e,0.3279,0,0.00%
acac_e,EX_acac_e,382.3,4,31.90%
ca2_e,EX_ca2_e,0.1093,0,0.00%
cl_e,EX_cl_e,0.1093,0,0.00%
cobalt2_e,EX_cobalt2_e,0.1093,0,0.00%
cu2_e,EX_cu2_e,0.1093,0,0.00%
etoh_e,EX_etoh_e,131.4,2,5.48%
glyc3p_e,EX_glyc3p_e,1000,3,62.59%
k_e,EX_k_e,0.1093,0,0.00%


In [183]:
M_xanthus.reactions.EX_o2_e.summary()

In [15]:
M_xanthus.metabolites.arg_L_c
M_xanthus.metabolites.asn_L_c
M_xanthus.metabolites.asp_L_c
M_xanthus.metabolites.gln_L_c
M_xanthus.metabolites.phe_L_c
M_xanthus.metabolites.ser_L_c
M_xanthus.metabolites.trp_L_c
M_xanthus.metabolites.tyr_L_c

Metabolite identifier,tyr_L_c
Name,L-Tyrosine [c]
Memory address,0x7759d4f21770
Formula,C9H11NO3
Compartment,c
In 6 reaction(s),"rxn04989_c, rxn01315_c, rxn00527_c, OF_PROTEIN, rxn12847_c, rxn09310_c"


In [188]:
E_coli.reactions.BIOMASS_Ec_iML1515_WT_75p37M.reaction

'0.000223 10fthf_c + 0.000223 2dmmql8_c + 2.5e-05 2fe2s_c + 0.000248 4fe4s_c + 0.000223 5mthf_c + 0.000279 accoa_c + 0.000223 adocbl_c + 0.499149 ala__L_c + 0.000223 amet_c + 0.28742 arg__L_c + 0.234232 asn__L_c + 0.234232 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.004952 ca2_c + 0.000223 chor_c + 0.004952 cl_c + 0.002944 clpn160_p + 0.00229 clpn161_p + 0.00118 clpn181_p + 0.000168 coa_c + 2.4e-05 cobalt2_c + 0.008151 colipa_e + 0.129799 ctp_c + 0.000674 cu2_c + 0.088988 cys__L_c + 0.024805 datp_c + 0.025612 dctp_c + 0.025612 dgtp_c + 0.024805 dttp_c + 0.000223 enter_c + 0.000223 fad_c + 0.006388 fe2_c + 0.007428 fe3_c + 0.255712 gln__L_c + 0.255712 glu__L_c + 0.595297 gly_c + 0.154187 glycogen_c + 0.000223 gthrd_c + 0.209121 gtp_c + 70.028756 h2o_c + 0.000223 hemeO_c + 0.092056 his__L_c + 0.282306 ile__L_c + 0.18569 k_c + 0.437778 leu__L_c + 3e-06 lipopb_c + 0.333448 lys__L_c + 3.1e-05 malcoa_c + 0.149336 met__L_c + 0.008253 mg2_c + 0.000223 mlthf_c + 0.000658 mn2_c + 7e-06 mobd_c + 